In [1]:

!pip install -q --use-deprecated=legacy-resolver tflite-model-maker
!pip install -q pycocotools
!pip install -q opencv-python-headless==4.1.2.30
!pip uninstall -y tensorflow && pip install -q tensorflow==2.8.0

ERROR: Could not find a version that satisfies the requirement tflite-support>=0.4.2 (from tflite-model-maker) (from versions: 0.1.0a0.dev3, 0.1.0a0.dev4, 0.1.0a0.dev5, 0.1.0a0, 0.1.0a1)
ERROR: No matching distribution found for tflite-support>=0.4.2 (from tflite-model-maker)
ERROR: Could not find a version that satisfies the requirement opencv-python-headless==4.1.2.30 (from versions: 3.4.10.37, 3.4.11.39, 3.4.11.41, 3.4.11.43, 3.4.11.45, 3.4.13.47, 3.4.15.55, 3.4.16.59, 3.4.17.61, 3.4.17.63, 3.4.18.65, 4.3.0.38, 4.4.0.40, 4.4.0.42, 4.4.0.44, 4.4.0.46, 4.5.1.48, 4.5.3.56, 4.5.4.58, 4.5.4.60, 4.5.5.62, 4.5.5.64, 4.6.0.66, 4.7.0.68, 4.7.0.72)
ERROR: No matching distribution found for opencv-python-headless==4.1.2.30


In [ ]:
!pip install pexpect 

: 

In [3]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)
#run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

/home/kevin/.local/lib/python3.8/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
/home/kevin/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below 2.13.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make 

In [4]:
TRAINING = 'annotations_train.csv'
VALIDATION = 'annotations_validation.csv'
TEST = 'annotations_test.csv'

In [3]:
spec = model_spec.get('efficientdet_lite0')

In [5]:
train_data, _validation_data, _test_data = object_detector.DataLoader.from_csv(TRAINING)
_train_data, validation_data, _test_data = object_detector.DataLoader.from_csv(VALIDATION)
_train_data, _validation_data, test_data = object_detector.DataLoader.from_csv(TEST)

In [5]:
print('Training data size:', train_data.size)
print('Validation data size:', validation_data.size)
print('Test data size:', test_data.size)

Training data size: 2503
Validation data size: 260
Test data size: 241


In [6]:
model = object_detector.create(train_data, model_spec=spec, batch_size=8, train_whole_model=True, validation_data=validation_data)

Epoch 1/50
312/312 [==============================] - 106s 254ms/step - det_loss: 0.7030 - cls_loss: 0.5017 - box_loss: 0.0040 - reg_l2_loss: 0.0638 - loss: 0.7668 - learning_rate: 0.0090 - gradient_norm: 1.8065 - val_det_loss: 1.0850 - val_cls_loss: 0.9977 - val_box_loss: 0.0017 - val_reg_l2_loss: 0.0638 - val_loss: 1.1488
Epoch 2/50
312/312 [==============================] - 78s 249ms/step - det_loss: 0.2690 - cls_loss: 0.1907 - box_loss: 0.0016 - reg_l2_loss: 0.0638 - loss: 0.3328 - learning_rate: 0.0100 - gradient_norm: 1.6276 - val_det_loss: 1.2156 - val_cls_loss: 1.1639 - val_box_loss: 0.0010 - val_reg_l2_loss: 0.0638 - val_loss: 1.2794
Epoch 3/50
312/312 [==============================] - 79s 252ms/step - det_loss: 0.2252 - cls_loss: 0.1572 - box_loss: 0.0014 - reg_l2_loss: 0.0638 - loss: 0.2890 - learning_rate: 0.0099 - gradient_norm: 1.5380 - val_det_loss: 1.3940 - val_cls_loss: 1.3531 - val_box_loss: 8.1707e-04 - val_reg_l2_loss: 0.0638 - val_loss: 1.4578
Epoch 4/50
312/312 [

In [7]:
model.export(export_dir='.', export_format=[ExportFormat.SAVED_MODEL, ExportFormat.LABEL])

2023-04-20 11:05:54.983453: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [8]:
model.export(export_dir='.', tflite_filename='model.tflite', quantization_config=QuantizationConfig.for_float16(), export_format=[ExportFormat.TFLITE])

2023-04-20 11:07:46.902312: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.


Estimated count of arithmetic ops: 1.767 G  ops, equivalently 0.883 G  MACs


2023-04-20 11:07:52.361831: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-04-20 11:07:52.361859: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.


In [9]:
model.evaluate(test_data)

2023-04-20 11:08:31.895798: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.28GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2023-04-20 11:08:31.895838: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.28GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2023-04-20 11:08:31.920150: W tensorflow/core/common_runtime/bfc_allocator.cc:275] Allocator (GPU_0_bfc) ran out of memory trying to allocate 616.00MiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2023-04-20 11:08:31.920181: W tensorflow/core/kernels/gpu_utils.cc:50] Failed to alloc

ResourceExhaustedError: Graph execution error:

2 root error(s) found.
  (0) RESOURCE_EXHAUSTED:  OOM when allocating tensor with shape[64,96,160,160] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node blocks_1/tpu_batch_normalization/FusedBatchNormV3}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

	 [[GatherNd_112/_1050]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.

  (1) RESOURCE_EXHAUSTED:  OOM when allocating tensor with shape[64,96,160,160] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node blocks_1/tpu_batch_normalization/FusedBatchNormV3}}]]
Hint: If you want to see a list of a

In [2]:
#test model.tflite with test_set
from_file = model.create('model.tflite', 'labels.txt')
from_file.evaluate(test_data)

NameError: name 'model' is not defined

NameError: name 'model' is not defined